# Giorno 2

In [1]:
import os
os.chdir('./..')
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
_BASE_PATH = 'resources/dataset/originals'

from src import data
from src.preprocessing.distances import preprocess
import src.utility as utils
from time import time

In [2]:
#Caricamento all speeds
new_speeds=pd.read_csv(f'{_BASE_PATH}/New_traffic_testset/new_speeds_2019.csv')
old_speed_train=pd.read_csv(f'{_BASE_PATH}/speeds_train.csv.gz')
old_speed_test=pd.read_csv(f'{_BASE_PATH}/speeds_test.csv.gz')

In [3]:
print(new_speeds.shape)
new_speeds.head()

(408824, 9)


,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
0,578.0,2019-01-04 13:45:00,601,177.247832,15.942993,148.856,229.756,179.0,578_601
1,334.0,2019-01-13 09:30:00,461,258.725905,48.175639,166.654,393.174,63.0,334_461
2,324.0,2019-01-05 09:30:00,218,85.574222,26.824718,72.810,156.946,9.0,324_218
3,363.0,2019-01-04 14:45:00,218,190.672931,25.182581,150.474,254.026,58.0,363_218
4,134.0,2019-01-04 09:45:00,75,188.033416,19.326115,150.474,249.172,89.0,134_75


In [4]:
print(old_speed_train.shape)
old_speed_train.head()

(10832482, 9)


,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
0,0,2018-09-01 01:00:00,443,97.889000,1.144099,97.080,98.698,2,0_443
1,0,2018-09-01 03:15:00,443,154.788667,16.287509,137.530,169.890,3,0_443
2,0,2018-09-01 12:45:00,443,136.091778,10.759666,124.586,163.418,18,0_443
3,0,2018-09-01 19:15:00,443,136.916276,15.089653,114.878,169.890,58,0_443
4,0,2018-09-02 09:45:00,443,135.476385,23.125778,82.518,169.890,26,0_443


In [5]:
print(old_speed_test.shape)
old_speed_test.head()

(3488814, 9)


,KEY,DATETIME_UTC,KM,SPEED_AVG,SPEED_SD,SPEED_MIN,SPEED_MAX,N_VEHICLES,KEY_2
0,0,2018-12-01 05:45:00,443,149.462750,11.339183,137.530,186.070,24,0_443
1,0,2018-12-01 14:30:00,443,183.164204,18.459800,140.766,223.284,49,0_443
2,0,2018-12-02 01:30:00,443,172.317000,14.873284,161.800,182.834,2,0_443
3,0,2018-12-02 08:00:00,443,181.913414,21.211454,139.148,224.902,58,0_443
4,0,2018-12-03 06:00:00,443,155.443571,19.456223,121.350,218.430,84,0_443


In [6]:
def create_base_structure_hours():
    """
    Call to create the base structure it is a pd Dataframe composed as follow:
    KEY | DATETIME_UTC | KM
    it is usefull to do join with other dataframe
    in it there are all the DATETIME_UTC present both in train and test speeds.csv files
    """
    start = time()

    # define the base path where to save the base_structure
    _BASE_PATH = 'resources/dataset/preprocessed'

    # check if the folder exsist if not create it
    utils.check_folder(_BASE_PATH)

    speeds_train = old_speed_train
    speeds_test = old_speed_test
    speeds_new = new_speeds

    # create all the datetimes between min train and max test datetime
    min_train_datetime = sorted(pd.to_datetime(speeds_train['DATETIME_UTC']).unique())[0].astype('int') // 10 ** 9
    max_test_datetime = sorted(pd.to_datetime(speeds_new['DATETIME_UTC']).unique())[-1].astype('int') // 10 ** 9


    
    range_datetimes = np.arange(min_train_datetime, max_test_datetime, 60 * 60)
    datetime_df = pd.DataFrame(pd.to_datetime(range_datetimes, unit='s'), columns=['DATETIME_UTC'])

    key_2_train = speeds_train.KEY_2.unique()
    key_2_test = speeds_test.KEY_2.unique()
    key_2_new = new_speeds.KEY_2.unique()

    # get all the unique key_2 in train and test
    key_2_full = sorted(set(key_2_test) | set(key_2_train)|set(key_2_new) )

    temp = pd.DataFrame(list(map(lambda x: x.split('_'), key_2_full)), columns=['KEY', 'KM'])

    # add dummy column to let a merge do a cartesian product
    temp['dummy'] = 0
    datetime_df['dummy'] = 0

    print('Doing cartesian product... it will take a while!')
    base_structure = pd.merge(datetime_df, temp).drop(['dummy'], axis=1)
    print('Done\n')

    print('sorting values...')
    base_structure = base_structure.sort_values(['DATETIME_UTC', 'KEY', 'KM']).reset_index(drop=True)
    print('Done\n')

    # save the base structure
    print('Saving base structure to {}/NEW_base_structure.csv'.format(_BASE_PATH))
    base_structure.to_csv(f'{_BASE_PATH}/NEW_base_structure_hours.csv', index=False)
    print('Done\n')

    print(f'PROCEDURE ENDED SUCCESSFULLY IN: {round(time() - start, 4)} s')

In [7]:
create_base_structure_hours()

Doing cartesian product... it will take a while!
Done

sorting values...
Done

Saving base structure to resources/dataset/preprocessed/NEW_base_structure.csv
Done

PROCEDURE ENDED SUCCESSFULLY IN: 24.5764 s


In [8]:
base_structure=pd.read_csv(f'{_BASE_PATH}/../preprocessed/base_structure_hours.csv')

base_structure.shape

(5298840, 3)

In [9]:
new_base_structure=pd.read_csv(f'{_BASE_PATH}/../preprocessed/NEW_base_structure_hours.csv')
new_base_structure['DATETIME_UTC'] = pd.to_datetime(new_base_structure.DATETIME_UTC)
new_base_structure.shape

print('new BS has ', new_base_structure.shape[0]-base_structure.shape[0], ' row more than the previuos one')

new BS has  1849329  row more than the previuos one


In [10]:
#slice della nuova bs con il solo nuovo periodo, risetto index ok????????

new_bs= new_base_structure.loc[new_base_structure['DATETIME_UTC']> '2018-12-31 23:45:00'].reset_index()
new_bs = new_bs.drop(["index"],axis=1)
print(new_bs.shape)
new_bs

(1874841, 3)


,DATETIME_UTC,KEY,KM
0,2019-01-01 00:00:00,0,333
1,2019-01-01 00:00:00,0,337
2,2019-01-01 00:00:00,0,342
3,2019-01-01 00:00:00,0,343
4,2019-01-01 00:00:00,0,357
5,2019-01-01 00:00:00,0,385
6,2019-01-01 00:00:00,0,407
7,2019-01-01 00:00:00,0,411
8,2019-01-01 00:00:00,0,423
9,2019-01-01 00:00:00,0,443


In [11]:
preprocess()
distances= data.distances_proprocessed()
distances

Preprocessing distances...
caching distances

distances loaded in: 0.0786 s

shape of the dataframe is: (41054, 2)
caching distances

distances loaded in: 0.1109 s

shape of the dataframe is: (41046, 3)


,STATIONS,KEY,KM
0,"STATION_29,10.0;STATION_37,19.0;STATION_36,40....",278,662
1,"STATION_29,10.0;STATION_37,19.0;STATION_36,40....",278,663
2,"STATION_29,10.0;STATION_37,19.0;STATION_36,40....",278,664
3,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,665
4,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,666
5,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,667
6,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,668
7,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,669
8,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,670
9,"STATION_29,6.0;STATION_37,17.0;STATION_36,44.0...",278,671


Il weather non contiene le seguenti stations !!! ['STATION_79', 'STATION_53', 'STATION_3']

prova = pd.DataFrame({"ID":['STATION_79', 'STATION_53', 'STATION_3'], 
                    "TEMPERATURE":[np.NaN, np.NaN, np.NaN],
                     "MAX_TEMPERATURE":[np.NaN, np.NaN, np.NaN],
                     "MIN_TEMPERATURE":[np.NaN, np.NaN, np.NaN],
                     "WEATHER":[np.NaN, np.NaN, np.NaN],}) 

In [12]:
new_weather_df = pd.read_csv(f'{_BASE_PATH}/New_traffic_testset/new_weather_2019.csv')
new_weather_df['DATETIME_UTC'] = pd.to_datetime(new_weather_df.DATETIME_UTC)
print(new_weather_df.dtypes)
new_weather_df.head(5)

ID                         object
TEMPERATURE               float64
MAX_TEMPERATURE           float64
MIN_TEMPERATURE           float64
WEATHER                    object
DATETIME_UTC       datetime64[ns]
dtype: object


,ID,TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE,WEATHER,DATETIME_UTC
0,STATION_8,10.1,14.2,6.8,Quasi Sereno,2019-01-10 19:00:00
1,STATION_83,5.5,5.5,1.9,Debole Neve,2019-02-03 13:00:00
2,STATION_11,13.8,15.5,6.6,Quasi Sereno,2019-01-04 18:00:00
3,STATION_15,17.8,17.8,14.8,Quasi Sereno,2019-01-07 13:00:00
4,STATION_77,12.5,12.5,6.4,Quasi Sereno,2019-02-05 10:00:00


In [13]:
stations=new_weather_df[["ID"]].drop_duplicates()
stations["dummy"]=0

In [14]:
times=new_bs[["DATETIME_UTC"]].drop_duplicates()
times["dummy"]=0

In [15]:
bs_2=pd.merge(stations,times)
bs_2=bs_2.drop("dummy",axis=1)
bs_2.head(10)

,ID,DATETIME_UTC
0,STATION_8,2019-01-01 00:00:00
1,STATION_8,2019-01-01 01:00:00
2,STATION_8,2019-01-01 02:00:00
3,STATION_8,2019-01-01 03:00:00
4,STATION_8,2019-01-01 04:00:00
5,STATION_8,2019-01-01 05:00:00
6,STATION_8,2019-01-01 06:00:00
7,STATION_8,2019-01-01 07:00:00
8,STATION_8,2019-01-01 08:00:00
9,STATION_8,2019-01-01 09:00:00


In [16]:
new_weather_df2=pd.merge(bs_2,new_weather_df,how="left")
new_weather_df2.head(10)

,ID,DATETIME_UTC,TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE,WEATHER
0,STATION_8,2019-01-01 00:00:00,6.5,12.6,0.9,Quasi Sereno
1,STATION_8,2019-01-01 01:00:00,6.2,6.2,6.2,Quasi Sereno
2,STATION_8,2019-01-01 02:00:00,6.5,6.5,6.2,Quasi Sereno
3,STATION_8,2019-01-01 03:00:00,6.3,6.5,6.2,Quasi Sereno
4,STATION_8,2019-01-01 04:00:00,6.2,6.5,6.2,Quasi Sereno
5,STATION_8,2019-01-01 05:00:00,5.5,6.5,5.5,Quasi Sereno
6,STATION_8,2019-01-01 06:00:00,5.8,6.5,5.5,Quasi Sereno
7,STATION_8,2019-01-01 07:00:00,5.9,6.5,5.5,Quasi Sereno
8,STATION_8,2019-01-01 08:00:00,6.2,6.5,5.5,Quasi Sereno
9,STATION_8,2019-01-01 09:00:00,7.9,7.9,5.5,Quasi Sereno


In [17]:
from tqdm.auto import tqdm


#Creazione Dizionario per le distanze
distances_dict = dict()
d=list(distances.groupby(by=["KEY","KM"]).groups.keys())
for x in tqdm(d):
    distances_dict[x] =list(map(lambda x: x.split(','), distances.loc[(distances['KEY']==x[0]) & 
                                                             (distances['KM']==x[1])].STATIONS.values[0].split(";")))


HBox(children=(IntProgress(value=0, max=41046), HTML(value='')))

In [18]:
new_weather_df2=new_weather_df2.set_index(["ID","DATETIME_UTC"])
new_weather_df2.sort_index(inplace=True)# This makes the lookup faster

In [20]:
import math, time

new_bs ["WEATHER"]=""
new_bs ["DISTANCE"]=np.NaN

new_bs ["TEMPERATURE"]=np.NaN
new_bs ["MAX_TEMPERATURE"]=np.NaN
new_bs ["MIN_TEMPERATURE"]=np.NaN
avoid_l=['STATION_79', 'STATION_53', 'STATION_3']



for i,row in tqdm(new_bs.iterrows()):
    date=row.DATETIME_UTC
    
    stations=distances_dict[(str(row.KEY),row.KM)]
    L = len(stations)
    weather = np.NaN
    it=0
    while not isinstance(weather, str):
        #get station and dist from dict
        station=stations[it][0]
        
        if station not in avoid_l:
            distance=stations[it][1]
            start_w = time.time()

            sl=new_weather_df2.loc[(station,date)]
            weather=sl.WEATHER.values[0]
            tmp=sl.TEMPERATURE.values[0]
            max_tmp=sl.MAX_TEMPERATURE.values[0]
            min_tmp=sl.MIN_TEMPERATURE.values[0]
            it+=1
            if it>= L:
                break
        else :
            it+=1
            if it>= L:
                break 
            
    new_bs.at[i,'WEATHER'] = weather
    new_bs.at[i,'TEMPERATURE'] = tmp
    new_bs.at[i,'MAX_TEMPERATURE'] = max_tmp
    new_bs.at[i,'MIN_TEMPERATURE'] = min_tmp
    new_bs.at[i,'DISTANCE'] = distance

new_bs.to_csv('resources/dataset/preprocessed/NEW_base_structure_df_weather.csv.gz', index=True, compression='gzip')
new_bs = pd.read_csv('resources/dataset/preprocessed/NEW_base_structure_df_weather.csv.gz', index_col=0, compression='gzip')
new_bs.head()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,DATETIME_UTC,KEY,KM,WEATHER,DISTANCE,TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
0,2019-01-01 00:00:00,0,333,Quasi Sereno,33.0,8.5,15.4,8.2
1,2019-01-01 00:00:00,0,337,Quasi Sereno,38.0,8.5,15.4,8.2
2,2019-01-01 00:00:00,0,342,Quasi Sereno,43.0,8.5,15.4,8.2
3,2019-01-01 00:00:00,0,343,Quasi Sereno,44.0,8.5,15.4,8.2
4,2019-01-01 00:00:00,0,357,Quasi Sereno,57.0,8.5,15.4,8.2


In [28]:
new_bs.loc[new_bs["WEATHER"].isna()==True]

,DATETIME_UTC,KEY,KM,WEATHER,DISTANCE,TEMPERATURE,MAX_TEMPERATURE,MIN_TEMPERATURE
19811,2019-01-01 11:00:00,0,333,NaN,221.0,NaN,NaN,NaN
19812,2019-01-01 11:00:00,0,337,NaN,226.0,NaN,NaN,NaN
19813,2019-01-01 11:00:00,0,342,NaN,231.0,NaN,NaN,NaN
19814,2019-01-01 11:00:00,0,343,NaN,232.0,NaN,NaN,NaN
19815,2019-01-01 11:00:00,0,357,NaN,245.0,NaN,NaN,NaN
19816,2019-01-01 11:00:00,0,385,NaN,268.0,NaN,NaN,NaN
19817,2019-01-01 11:00:00,0,407,NaN,286.0,NaN,NaN,NaN
19818,2019-01-01 11:00:00,0,411,NaN,279.0,NaN,NaN,NaN
19819,2019-01-01 11:00:00,0,423,NaN,270.0,NaN,NaN,NaN
19820,2019-01-01 11:00:00,0,443,NaN,262.0,NaN,NaN,NaN


In [24]:
#Stesso script di prima eseguito in maniera parallela
from multiprocessing import Pool, Value
import sys, time

num_partitions = 10 #number of partitions to split dataframe
num_cores = os.cpu_count() #number of cores on your machine
counter = None

def init(args):
    ''' store the counter for later use '''
    global counter
    counter = args
    
def parallelize_dataframe(df, func):
    df_split = np.array_split(df, num_partitions)
    pool = Pool(num_cores, initializer = init, initargs = (counter, ))
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

def parse_dataset(df):
    for i,row in df.iterrows():
        with counter.get_lock():
            counter.value += 1
            if divmod(counter.value, 100)[1] == 0:
                print(f'{counter.value} rows processed', end='\r')
            sys.stdout.flush()
        date=row.DATETIME_UTC
        stations=distances_dict[(str(row.KEY),row.KM)]
        L = len(stations)
        weather = np.NaN
        it=0
        while not isinstance(weather, str):
            #get station and dist from dict
            station=stations[it][0]
            distance=stations[it][1]
            weather=weather_train_df2.loc[(station,date)].WEATHER.values[0]
            it+=1
            if it>= L:
                break

        df.at[i,'WEATHER'] = weather
        df.at[i,'DISTANCE'] = distance
    return df

base_structure_df ["WEATHER"]=""
base_structure_df ["DISTANCE"]=np.NaN

counter = Value('i', 0)  
start = time.time()
base_structure_df = parallelize_dataframe(base_structure_df, parse_dataset)
print(f'Time elapsed {time.time()-start}')

NameError: name 'base_structure_df' is not defined

In [14]:
base_structure_df.to_csv('resources/dataset/preprocessed/base_structure_df_weather.csv.gz', index=True, compression='gzip')

In [14]:
base_structure_df = pd.read_csv('resources/dataset/preprocessed/base_structure_df_weather.csv.gz', index_col=0, compression='gzip')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [15]:
base_structure_df.head()

,DATETIME_UTC,KEY,KM,WEATHER,DISTANCE
0,2018-09-01 00:00:00,0,333,Quasi Sereno,33.0
1,2018-09-01 00:00:00,0,337,Quasi Sereno,38.0
2,2018-09-01 00:00:00,0,342,Quasi Sereno,43.0
3,2018-09-01 00:00:00,0,343,Quasi Sereno,44.0
4,2018-09-01 00:00:00,0,357,Quasi Sereno,57.0
